In [44]:
import numpy as np
from bokeh.plotting import output_notebook, Figure, show
from bokeh.resources import INLINE
from bokeh.models import Column, ColumnDataSource, Range1d, Row, Circle, HoverTool, Div, Spacer

np.set_printoptions(suppress=True, precision=3)
output_notebook(resources=INLINE)

Loading BokehJS ...

In [38]:

def scatter_matrix(design, k_factors=None, width=None, height=None, margin=2, size=8, alpha=1, all_range=None, source=None):
    if k_factors is None:
        k_factors = len(design[0, :]) if type(design) is np.ndarray else len(design)
        k_factors = k_factors if k_factors <= 10 else 10
    
    width = 1000 if width is None else width
    width = width // k_factors - margin
    height = 800 if height is None else height
    height = height // k_factors - margin
    
    if source is None or k_factors > 2:
        fsource = ColumnDataSource(data=dict([(str(i), design[:, i]) for i in range(k_factors)]))

    colors = np.arange(k_factors, k_factors * 14 // 5)
    colors = np.vstack([np.random.choice(colors, 3) for i in range(k_factors)]) / k_factors / 2
    
    grey = (234, 234, 234)
    sgrey = np.array(grey) / 255
    
    if k_factors == 2:
        color = (colors[0, :] * colors[1, :] * 128).astype(int)
        scolor = color / 255
        color = tuple(color)
        md = (scolor - sgrey).sum()
        light_color = tuple(((scolor ** 4 ** md) * 255).astype(int))
        g = Figure(width=width * 2, height=height * 2, tools='box_select,wheel_zoom', toolbar_location=None)
        if source is not None:
            glyph = g.circle(x=design[0], y=design[1], source=source, color=color, line_color=None, size=size, alpha=alpha)
            
            glyph.selection_glyph = Circle(fill_alpha=np.sqrt(alpha), fill_color=color, line_color=None)
            glyph.nonselection_glyph = Circle(fill_alpha=1, fill_color='#EAEAEA', line_color=light_color,
                                              line_alpha=1, line_width=size/4)
        else:
            g.circle(x=str(0), y=str(1), source=fsource, color=color, line_color=None, size=size, alpha=alpha)
        
        if all_range is not None:
            g.y_range = Range1d(*all_range)
            g.x_range = Range1d(*all_range)
        return g

    rows = []
    for i in range(k_factors):
        row = []
        for j in range(k_factors):
            #if i == j:
            #    continue
            if i <= j:
                color = (colors[i, :] * colors[j, :] * 128).astype(int)
                scolor = color / 255
                color = tuple(color)
                md = (scolor - sgrey).sum()
                light_color = tuple(((scolor ** 4 ** md) * 255).astype(int))
                g = Figure(width=width, height=height, tools='box_select,wheel_zoom', toolbar_location=None)
                glyph = g.circle(x=str(j), y=str(i), source=fsource, color=color, line_color=None, size=size, alpha=alpha)
                glyph.selection_glyph = Circle(fill_alpha=np.sqrt(alpha), fill_color=color, line_color=None)
                glyph.nonselection_glyph = Circle(fill_alpha=1, fill_color='#EAEAEA', line_color=light_color,
                                                  line_alpha=1, line_width=size/4)

                g.xaxis.visible = False
                g.yaxis.visible = False
                if all_range is not None:
                    g.y_range = Range1d(*all_range)
                    g.x_range = Range1d(*all_range)
                row.append(g)
            else:
                row.append(Spacer(width=width))
        rows.append(Row(*row))

    return Column(*rows)

def cc_v_bb_comparison(n=4, width=800, alpha=0.6, size=10, face='ccf'):
    w = (width - 40) // 2
    h = width // 2
    cc = ccdesign(n, face=face)
    all_range = (-1.5, 1.5) if face is 'ccf' else None
    cc_g = scatter_matrix(cc, size=size, height=h, width=w, alpha=alpha, all_range=all_range)

    bb = bbdesign(n)
    bb_g = scatter_matrix(bb, size=size, height=h, width=w, alpha=alpha,  all_range=(-1.5, 1.5))

    inner_text = ('<h1 style="text-align:center;">{}</h1>'
                  + '<h3 style="text-align:center;margin-top:5px;color:#7C7C7C;">{:d} Factors : {:d} Experiments</h3>')

    return Column(Row(Div(width=w, height=40, text=inner_text.format('Central-Composite', n, len(cc))),
                      Spacer(width=40),
                      Div(width=w, height=40, text=inner_text.format('Box-Behnken', n, len(bb)))),
                  Row(cc_g, Spacer(width=40), bb_g))

# experimentspydesign
## Demonstration
#### Josh McCrary

# experimentspydesign
## Main Features

Only depends on numpy

Factor classes use binary operaters to create factorial combinations

Contains simple design methods with selectable scaling

Factor classes expand scaling interface and enable more explicit and persistent definitions for particular collections of factors for a design

# Notes/Comments/Quirks
Returned tables from design methods are always of type float since they are numpy arrays and can hold equivalent int values.

# Interface Overview
Provide the number of factors in the design (excluding fullfact) and define the default scale:

    method(number_of_factors, def_scale='traditional', **kwargs)

Submit an iterable or Factor class for each factor with the levels for the factor and/or an integer to define the number of levels:

    method((-10, 100), 7, np.arange(5, 14) **2, 
           def_scale='standard', **kwargs)
Same as:
    
    method([(-10, 100), 7, np.arange(5, 14) **2], 
           def_scale='standard', **kwargs)

Factors with explicit definitions will be scaled to their definitions and only factors with the number of levels defined will follow the `def_scale` paradigm.


# Import design methods

In [2]:
from experimentspydesign import ff2n, fullfact, bbdesign, ccdesign, lhs

# ff2n
#### Only 2 levels for each factor
Every combination of each factor's high and low values, design with $ 2 ^ k $ experiments.

In [45]:
switch = False
factors = ([3, 4], [.001, .01, .1, 1.0], [42, 1138]) if switch else 3
des = ff2n(factors, def_scale='traditional')
print(len(des), 'experiments:')
print(des)

8 experiments:
[[-1 -1 -1]
 [ 1 -1 -1]
 [-1  1 -1]
 [ 1  1 -1]
 [-1 -1  1]
 [ 1 -1  1]
 [-1  1  1]
 [ 1  1  1]]


# fullfact
#### NOTE impossible to define a full factorial design with just a single integer
To make a full factorial design, the minimum information needed is the number of levels for each factor.
#### Number of levels for each factor is the length of the iterable or the integer passed for the factor
Every combination of each factor's defined levels, design with $\prod _{i=1}^{k} n_i $ experiments, where $ n_i $ is the number of levels for factor $ i $.

In [46]:
f_n_levels = [3, 2, 4]
factors = ([3, 42, 1138], [.001, .01, .1, 1.0])
des = fullfact(f_n_levels, def_scale='standard')
print(len(des), 'experiments:')
print(des)

24 experiments:
[[ 0.     0.     0.   ]
 [ 0.5    0.     0.   ]
 [ 1.     0.     0.   ]
 [ 0.     1.     0.   ]
 [ 0.5    1.     0.   ]
 [ 1.     1.     0.   ]
 [ 0.     0.     0.333]
 [ 0.5    0.     0.333]
 [ 1.     0.     0.333]
 [ 0.     1.     0.333]
 [ 0.5    1.     0.333]
 [ 1.     1.     0.333]
 [ 0.     0.     0.667]
 [ 0.5    0.     0.667]
 [ 1.     0.     0.667]
 [ 0.     1.     0.667]
 [ 0.5    1.     0.667]
 [ 1.     1.     0.667]
 [ 0.     0.     1.   ]
 [ 0.5    0.     1.   ]
 [ 1.     0.     1.   ]
 [ 0.     1.     1.   ]
 [ 0.5    1.     1.   ]
 [ 1.     1.     1.   ]]


# lhs 
## Latin Hypercube Sampling
Choose the number of experiments in the design $ n $, each factor will have $ n $ levels evenly spaced from the low to high levels. 

If a factor has the same number of levels, $ n $, explicitly defined it will have those values in the final design rather than a linear spacing across the low to high values.

In [47]:
n = 11
factors = (1, 1, [(i - 6) * abs(i - 6) for i in range(n)]) if n == 10 else 3
des = lhs(factors, n_samples=n, def_scale='level_n')
print(len(des), 'experiments:')
print(des)

11 experiments:
[[ 2  9  2]
 [ 3  3  4]
 [11  0 11]
 [ 5  7  3]
 [ 9  8  9]
 [ 8  4  0]
 [ 0  6  5]
 [ 6 11  8]
 [ 7  1  6]
 [ 4  2  7]
 [ 1  5  1]]


# ccdesign
## Central Composite

Design used for quadratic models of a respone expands ff2n design by adding *star points on the hypersphere(ball) or hypercube of the design space.
Central Composite designs have $ 2^k + 2k + n_c $ experiments, where $n_c \geq 1$ is a number of additional points added at the origin for variance 

#### Circumscribed
The *star points which are added to the ff2n design are scaled outside of the factor's level range to maintain rotatability or some amount of orthogonality in the design. 

#### Inscribed
The *star points are placed on the center of the face of each side of the k-dimensional cube and the corners are scaled down to maintain rotatability or some amount of orthogonality in the design.

#### Face
The *star points are placed on the center of the face of each side of the k-dimensional cube and the corners retain their low and high values, this design is not rotatable nor orthogonal

In [22]:
ccc = ccdesign(3, 3, 3, face='ccc') # default face paradigm
cci = ccdesign(3, 3, 3, face='cci')
ccf = ccdesign(3, 3, 3, face='ccf')
print(len(ccc), 'experiments:')
print(ccc[:15, :])

19 experiments:
[[-1.         -1.         -1.        ]
 [ 1.         -1.         -1.        ]
 [-1.          1.         -1.        ]
 [ 1.          1.         -1.        ]
 [-1.         -1.          1.        ]
 [ 1.         -1.          1.        ]
 [-1.          1.          1.        ]
 [ 1.          1.          1.        ]
 [ 1.68179283  0.          0.        ]
 [ 0.          1.68179283  0.        ]
 [ 0.          0.          1.68179283]
 [-1.68179283  0.          0.        ]
 [ 0.         -1.68179283  0.        ]
 [ 0.          0.         -1.68179283]
 [ 0.          0.          0.        ]]


# bbdesign
## Box-Behnken

Similar to central composite and used for quadratic models, but more efficient for larger number of factors and maintains good rotatability.

In [21]:
bb = bbdesign(3, 3, 3, n_centers=4)
print(len(bb), 'experiments:')
print(bb[:15, :])

16 experiments:
[[-1 -1  0]
 [ 1 -1  0]
 [-1  1  0]
 [ 1  1  0]
 [-1  0 -1]
 [ 1  0 -1]
 [-1  0  1]
 [ 1  0  1]
 [ 0 -1 -1]
 [ 0  1 -1]
 [ 0 -1  1]
 [ 0  1  1]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]]


In [40]:
cc_v_bb = cc_v_bb_comparison(n=7, width=900, size=7, alpha=0.5)
show(cc_v_bb)

In [17]:
from time import time
n_factors = np.arange(19) + 3
bb_n = []
cc_n = []
bb_t = []
cc_t = []
for k in n_factors:
    t0 = time()
    bb = bbdesign(int(k))
    t1 = time()
    ccc = ccdesign(int(k))
    t2 = time()
    bb_n.append(len(bb))
    print('Box-Behnken, {:d} factors create {:d} experiment combinations'.format(*bb.shape[::-1]))
    bb_t.append(t1 - t0)
    print('  time elapsed for {:d} factors:'.format(k), bb_t[-1])
    cc_n.append(len(ccc))
    print('Central Composite, {:d} factors create {:d} experiment combinations'.format(*ccc.shape[::-1]))
    cc_t.append(t2 - t1)
    print('  time elapsed for {:d} factors:'.format(k), cc_t[-1])
bb_n = np.array(bb_n)
cc_n = np.array(cc_n)

Box-Behnken, 3 factors create 17 experiment combinations
  time elapsed for 3 factors: 0.0010025501251220703
Central Composite, 3 factors create 19 experiment combinations
  time elapsed for 3 factors: 0.0010025501251220703
Box-Behnken, 4 factors create 29 experiment combinations
  time elapsed for 4 factors: 0.0010023117065429688
Central Composite, 4 factors create 30 experiment combinations
  time elapsed for 4 factors: 0.0010030269622802734
Box-Behnken, 5 factors create 45 experiment combinations
  time elapsed for 5 factors: 0.0010025501251220703
Central Composite, 5 factors create 48 experiment combinations
  time elapsed for 5 factors: 0.0
Box-Behnken, 6 factors create 66 experiment combinations
  time elapsed for 6 factors: 0.0
Central Composite, 6 factors create 82 experiment combinations
  time elapsed for 6 factors: 0.0010030269622802734
Box-Behnken, 7 factors create 90 experiment combinations
  time elapsed for 7 factors: 0.0010025501251220703
Central Composite, 7 factors cr

In [18]:
source = ColumnDataSource(data={'k':n_factors[:22], 'bb':bb_n, 'cc':cc_n, 'ratio':np.round(bb_n / cc_n, decimals=3)})
f = Figure(width=600, height=180, tools='box_select', toolbar_location=None)
f.circle('k', 'bb', color='orange', size=6, source=source, legend='Box-Behnken', alpha=0.7)
f.circle('k', 'cc', size=6, source=source, legend='Central-Composite', alpha=0.7)
f.add_tools(HoverTool(tooltips=[('k factors', '@k'), ('bb', '@bb'), ('cc', '@cc'), ('ratio', '@ratio')]))

In [19]:
width = 200
height = 180
size = 12
bbsource = ColumnDataSource(data={'1':bb[:, 0], '2':bb[:, 1], '3':bb[:, 2]})
bb_12 = scatter_matrix(('1', '2'), width=width, height=height, size=size, alpha=0.5, source=bbsource)
bb_12.title.text = 'Box-Behnken_12'
bb_13 = scatter_matrix(('1', '3'), width=width, height=height, size=size, alpha=0.5, source=bbsource)
bb_13.title.text = 'Box-Behnken_13'
bb_23 = scatter_matrix(('2', '3'), width=width, height=height, size=size, alpha=0.5, source=bbsource)
bb_23.title.text = 'Box-Behnken_23'

ccsource = ColumnDataSource(data={'ccc_x':ccc[:, 0],
                                  'ccc_y':ccc[:, 1],
                                  'cci_x':cci[:, 0],
                                  'cci_y':cci[:, 1],
                                  'ccf_x':ccf[:, 0],
                                  'ccf_y':ccf[:, 1]})
ccc_g = scatter_matrix(('ccc_x', 'ccc_y'), width=width, height=height, size=size, alpha=0.5, source=ccsource)
ccc_g.title.text = 'Circumscribed_12'
cci_g = scatter_matrix(('cci_x', 'cci_y'), width=width, height=height, size=size, alpha=0.5, source=ccsource)
cci_g.title.text = 'Inscribed_12'
ccf_g = scatter_matrix(('ccf_x', 'ccf_y'), width=width, height=height, size=size, alpha=0.5, source=ccsource)
ccf_g.title.text = 'On Face_12'

for p in [bb_12, bb_13, bb_23, ccc_g, cci_g, ccf_g]:
    p.x_range = Range1d(-1.8, 1.8)
    p.y_range = Range1d(-1.8, 1.8)

# bbdesign vs ccdesign

In [20]:
show(Column(Row(bb_12, bb_13, bb_23), Row(ccc_g, cci_g, ccf_g), f))

In [32]:
des = ff2n(6)
g = scatter_matrix(des, width=600, height=400, alpha=0.5, size=8, all_range=(-1.5, 1.5))
show(g)

In [85]:
des = fullfact(2, 3, 4, 5)
g = scatter_matrix(des, width=600, height=400, alpha=0.5, size=12, all_range=(-1.5, 1.5))
print(des.shape)
show(g)

(120, 4)


In [117]:
from experimentspydesign import FactorDiscrete, FactorContinuous, Design

In [120]:
des = fullfact(2, 3, 
               FactorContinuous((np.arange(1, 5) ** 2 / 8 - 1).tolist()),
               FactorContinuous((np.arange(-2, 3) ** 3 / 8).tolist()))
g = scatter_matrix(des, width=600, height=400, alpha=0.5, size=12, all_range=(-1.5, 1.5))
print(des.shape)
show(g)

(120, 4)


In [37]:
des = bbdesign(3, 3, 3, 3)
des_g = scatter_matrix(des, width=600, height=400, size=12, alpha=0.5, all_range=(-1.5, 1.5))
show(des_g)

In [121]:
f1 = FactorDiscrete(1, 10, 100)
f2 = FactorContinuous(4, 4.5, n_levels=5)
fc = f1 * f2

In [122]:
fc

    factor_05__factor_06
 0   [1, [4.0, 4.1]] 
 1   [1, [4.1, 4.2]] 
 2   [1, [4.2, 4.3]] 
 3   [1, [4.3, 4.4]] 
 4   [1, [4.4, 4.5]] 
 5  [10, [4.0, 4.1]] 
 6  [10, [4.1, 4.2]] 
 7  [10, [4.2, 4.3]] 
 8  [10, [4.3, 4.4]] 
 9  [10, [4.4, 4.5]] 
10  [100, [4.0, 4.1]]
11  [100, [4.1, 4.2]]
12  [100, [4.2, 4.3]]
13  [100, [4.3, 4.4]]
14  [100, [4.4, 4.5]]

In [125]:
fc.levels

      factor_05__factor_06   
 0   [1, 4.088829427080821]  
 1   [1, 4.174518646693514]  
 2   [1, 4.268858671675487]  
 3   [1, 4.345211225302373]  
 4   [1, 4.413538003091516]  
 5   [10, 4.073430361751926] 
 6  [10, 4.1988110622017745] 
 7  [10, 4.2459001024504985] 
 8   [10, 4.399021208200122] 
 9   [10, 4.45732096367716]  
10  [100, 4.017267396653594] 
11  [100, 4.163947751643164] 
12  [100, 4.259446064747849] 
13  [100, 4.306620463504838] 
14  [100, 4.4587390095399755]

In [127]:
flower = FactorDiscrete([i * 10 + 30 for i in range(6)], name='lower')
fupper = FactorDiscrete([i * 10 + 40 for i in range(6)], name='upper')
finterval = flower < fupper
finterval

    lower__upper
 0  [30, 40]
 1  [30, 50]
 2  [30, 60]
 3  [30, 70]
 4  [30, 80]
 5  [30, 90]
 6  [40, 50]
 7  [40, 60]
 8  [40, 70]
 9  [40, 80]
10  [40, 90]
11  [50, 60]
12  [50, 70]
13  [50, 80]
14  [50, 90]
15  [60, 70]
16  [60, 80]
17  [60, 90]
18  [70, 80]
19  [70, 90]
20  [80, 90]

In [128]:
flower <= fupper

    lower__upper
 0  [30, 40]
 1  [30, 50]
 2  [30, 60]
 3  [30, 70]
 4  [30, 80]
 5  [30, 90]
 6  [40, 40]
 7  [40, 50]
 8  [40, 60]
 9  [40, 70]
...
16  [50, 90]
17  [60, 60]
18  [60, 70]
19  [60, 80]
20  [60, 90]
21  [70, 70]
22  [70, 80]
23  [70, 90]
24  [80, 80]
25  [80, 90]

In [129]:
finterval.levels

    lower__upper
 0  [30, 40]
 1  [30, 50]
 2  [30, 60]
 3  [30, 70]
 4  [30, 80]
 5  [30, 90]
 6  [40, 50]
 7  [40, 60]
 8  [40, 70]
 9  [40, 80]
10  [40, 90]
11  [50, 60]
12  [50, 70]
13  [50, 80]
14  [50, 90]
15  [60, 70]
16  [60, 80]
17  [60, 90]
18  [70, 80]
19  [70, 90]
20  [80, 90]

In [133]:
design = Design()
f0 = FactorDiscrete(3, 14, 42, 51)
print(f0.name)
design['distance'] = f0
print(f0.name)

factor_08
distance


In [135]:
design['stealth'] = FactorContinuous(5, 10, n_levels=5)
design['speed'] = FactorDiscrete(40, 60, n_levels=4)
design


distance                 distance
0  3 
1  14
2  42
3  51             
stealth       stealth  
0  [5.0, 6.0] 
1  [6.0, 7.0] 
2  [7.0, 8.0] 
3  [8.0, 9.0] 
4  [9.0, 10.0]
speed      speed  
0  [40, 44]
1  [45, 49]
2  [50, 54]
3  [55, 60] 

In [36]:
def cc_v_bb_comparison(n=4, width=800, alpha=0.6, size=10, face='ccf'):
    w = (width - 40) // 2
    h = width // 2
    cc = ccdesign(n, face=face)
    all_range = (-1.5, 1.5) if face is 'ccf' else None
    cc_g = scatter_matrix(cc, size=size, height=h, width=w, alpha=alpha, all_range=all_range)

    bb = bbdesign(n)
    bb_g = scatter_matrix(bb, size=size, height=h, width=w, alpha=alpha,  all_range=(-1.5, 1.5))

    inner_text = ('<h1 style="text-align:center;">{}</h1>'
                  + '<h3 style="text-align:center;margin-top:5px;color:#7C7C7C;">{:d} Factors : {:d} Experiments</h3>')

    return Column(Row(Div(width=w, height=40, text=inner_text.format('Central-Composite', n, len(cc))),
                      Spacer(width=40),
                      Div(width=w, height=40, text=inner_text.format('Box-Behnken', n, len(bb)))),
                  Row(cc_g, Spacer(width=40), bb_g))

In [112]:
len(bb6)

66

In [114]:
len(cc6)

82

In [41]:
from experimentspydesign import lhs

In [43]:
des = lhs([2] * 7, n_samples=149)
des_g = scatter_matrix(des, size=6, height=600, width=800)
show(des_g)

In [83]:
bb = bbdesign(4)
print(bb)

[[-1. -1.  0.  0.]
 [ 1. -1.  0.  0.]
 [-1.  1.  0.  0.]
 [ 1.  1.  0.  0.]
 [-1.  0. -1.  0.]
 [ 1.  0. -1.  0.]
 [-1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [-1.  0.  0. -1.]
 [ 1.  0.  0. -1.]
 [-1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 0. -1. -1.  0.]
 [ 0.  1. -1.  0.]
 [ 0. -1.  1.  0.]
 [ 0.  1.  1.  0.]
 [ 0. -1.  0. -1.]
 [ 0.  1.  0. -1.]
 [ 0. -1.  0.  1.]
 [ 0.  1.  0.  1.]
 [ 0.  0. -1. -1.]
 [ 0.  0.  1. -1.]
 [ 0.  0. -1.  1.]
 [ 0.  0.  1.  1.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [82]:
cc = ccdesign(3)
print(cc)

[[-1.         -1.         -1.        ]
 [ 1.         -1.         -1.        ]
 [-1.          1.         -1.        ]
 [ 1.          1.         -1.        ]
 [-1.         -1.          1.        ]
 [ 1.         -1.          1.        ]
 [-1.          1.          1.        ]
 [ 1.          1.          1.        ]
 [ 1.68179283  0.          0.        ]
 [ 0.          1.68179283  0.        ]
 [ 0.          0.          1.68179283]
 [-1.68179283  0.          0.        ]
 [ 0.         -1.68179283  0.        ]
 [ 0.          0.         -1.68179283]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]


In [15]:
[(i, j) for i in range(5) for j in range(5) if j > i]

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4)]

In [136]:
mydesign = Design()
mydesign['hardness'] = FactorDiscrete(4, 9)
mydesign['richter'] = FactorContinuous(3.2, 8.7)
mydesign['pressure'] = FactorContinuous(2800, 3300)
mydesign


hardness        hardness
0  4
1  9    
 richter       richter
0  3.2
1  8.7  
pressure     pressure
0  2800
1  3300 

In [137]:
lhs(mydesign, n_samples=20)

AttributeError: 'dict_values' object has no attribute 'lower'

In [148]:
for k in mydesign.keys():
    print(k, max([mydesign[k].values()]).upper)

AttributeError: 'dict_values' object has no attribute 'upper'